# Modelling: Stress_Level Prediction

**Target:** `Stress_Level` (Low/Medium/High)  
**Approach:** Train ≥2 models + dummy baseline. Select final model using **cross-validated macro-F1**, then evaluate once on test set.  
**Note:** This is predictive modelling (association), not causal inference.


In [52]:
#importing relevant libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

In [53]:
#loading the data and quick target checks
df = pd.read_csv("../data/Impact_of_Remote_Work_on_Mental_Health.csv")
print(f"Shape: {df.shape}")
print(f"\nColumns:\n {df.columns}")
print(f"\nTarget:\n {df["Stress_Level"].value_counts(normalize=True)}")
assert "Stress_Level" in df.columns
print("\nMissing target:", df["Stress_Level"].isna().sum())
print(df["Stress_Level"].value_counts())


Shape: (5000, 20)

Columns:
 Index(['Employee_ID', 'Age', 'Gender', 'Job_Role', 'Industry',
       'Years_of_Experience', 'Work_Location', 'Hours_Worked_Per_Week',
       'Number_of_Virtual_Meetings', 'Work_Life_Balance_Rating',
       'Stress_Level', 'Mental_Health_Condition',
       'Access_to_Mental_Health_Resources', 'Productivity_Change',
       'Social_Isolation_Rating', 'Satisfaction_with_Remote_Work',
       'Company_Support_for_Remote_Work', 'Physical_Activity', 'Sleep_Quality',
       'Region'],
      dtype='object')

Target:
 Stress_Level
High      0.3372
Medium    0.3338
Low       0.3290
Name: proportion, dtype: float64

Missing target: 0
Stress_Level
High      1686
Medium    1669
Low       1645
Name: count, dtype: int64


In [54]:
# Feature groups for preprocessing

ordinal_features = ["Work_Life_Balance_Rating", "Social_Isolation_Rating", "Satisfaction_with_Remote_Work", "Company_Support_for_Remote_Work", 
                    "Sleep_Quality"]
nominal_features = ["Work_Location", "Job_Role"]
numeric_features = ["Age", "Years_of_Experience", "Hours_Worked_Per_Week","Number_of_Virtual_Meetings"]

print(f"Ordinal: {ordinal_features}")
print(f"Nominal: {nominal_features}")
print(f"Numeric: {numeric_features}")

Ordinal: ['Work_Life_Balance_Rating', 'Social_Isolation_Rating', 'Satisfaction_with_Remote_Work', 'Company_Support_for_Remote_Work', 'Sleep_Quality']
Nominal: ['Work_Location', 'Job_Role']
Numeric: ['Age', 'Years_of_Experience', 'Hours_Worked_Per_Week', 'Number_of_Virtual_Meetings']


In [55]:
#unwanted columns from the data
drop_cols = ["Employee_ID", "Mental_Health_Condition", "Productivity_Change", "Physical_Activity","Gender",
            "Industry", "Region"]

#deleting the columns and updating df
df = df.drop(columns=drop_cols, errors="ignore")
print(f"Shape after drop: {df.shape}")
df.head()

Shape after drop: (5000, 13)


,Age,Job_Role,Years_of_Experience,Work_Location,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Stress_Level,Access_to_Mental_Health_Resources,Social_Isolation_Rating,Satisfaction_with_Remote_Work,Company_Support_for_Remote_Work,Sleep_Quality
0,32,HR,13,Hybrid,47,7,2,Medium,No,1,Unsatisfied,1,Good
1,40,Data Scientist,3,Remote,52,4,1,Medium,No,3,Satisfied,2,Good
2,59,Software Engineer,22,Hybrid,46,11,5,Medium,No,4,Unsatisfied,5,Poor
3,27,Software Engineer,20,Onsite,32,8,4,High,Yes,3,Unsatisfied,3,Poor
4,49,Sales,32,Onsite,35,12,2,High,Yes,3,Unsatisfied,3,Average


## Feature exclusion (leakage / non-predictive / scope)
- Dropped ID column (`Employee_ID`)
- Dropped likely downstream/leakage variables (e.g., `Productivity_Change`, possibly `Mental_Health_Condition`)
- Dropped some demographic/context fields to keep the model focused and reduce noise (can be revisited as future work)


In [56]:
#dropping the target from feature and defining feature 
X = df.drop(columns = ["Stress_Level"])

#defining the target (y)
y = df["Stress_Level"]

In [57]:
#evaluation
print(f"X shape: {X.shape}")
print(f"\ny distribution: \n {y.value_counts(normalize = True)}")

X shape: (5000, 12)

y distribution: 
 Stress_Level
High      0.3372
Medium    0.3338
Low       0.3290
Name: proportion, dtype: float64


In [58]:
#moving ordinal columns with numeric values to numeric features list
numeric_ordinals = [c for c in ordinal_features if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
numeric_features = list(dict.fromkeys(numeric_features + numeric_ordinals))
ordinal_features = [c for c in ordinal_features if c not in numeric_ordinals]

print(f" Numeric ordinals moved from ordinal features to numeric features: {numeric_ordinals}")
print(f"final numeric features: {numeric_features}")
print(f"final ordinal features: {ordinal_features}")
print(f"final nominal features: {nominal_features}")

 Numeric ordinals moved from ordinal features to numeric features: ['Work_Life_Balance_Rating', 'Social_Isolation_Rating', 'Company_Support_for_Remote_Work']
final numeric features: ['Age', 'Years_of_Experience', 'Hours_Worked_Per_Week', 'Number_of_Virtual_Meetings', 'Work_Life_Balance_Rating', 'Social_Isolation_Rating', 'Company_Support_for_Remote_Work']
final ordinal features: ['Satisfaction_with_Remote_Work', 'Sleep_Quality']
final nominal features: ['Work_Location', 'Job_Role']


In [60]:
# Numeric features pipeline
# Impute missing numeric values using the median
# (robust to outliers compared to mean)
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])


# Ordinal features pipeline
# Sleep_Quality has a natural order:
# Poor < Average < Good
# We explicitly define the order to prevent alphabetical encoding
ordinal_categories = [
    ["Poor", "Average", "Good"]   # Sleep_Quality
]

ord_pipe = Pipeline([
    # Impute missing ordinal values with most frequent category
    ("imputer", SimpleImputer(strategy="most_frequent")),
    
    # Encode categories using defined order
    # Unknown categories (if any) will be encoded as -1
    ("encoder", OrdinalEncoder(
        categories=ordinal_categories,
        handle_unknown="use_encoded_value",
        unknown_value=-1
    ))
])


# Nominal (unordered) categorical features pipeline
# Impute missing values with most frequent category
# Then apply one-hot encoding (no ordinal meaning assumed)
nom_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])


# Combine all preprocessing steps
# Each feature group is processed separately
# Output is a fully numeric matrix ready for modelling
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_features),
        ("ord", ord_pipe, ordinal_features),
        ("nom", nom_pipe, nominal_features),
    ],
    remainder="drop"  # Drop any columns not explicitly listed
)


### Encoding of Sleep_Quality

`Sleep_Quality` has ordered categories: Poor < Average < Good.

This variable is encoded using an ordinal encoder with an explicitly defined category order to preserve its natural ranking. Alphabetical encoding would produce an incorrect order and distort model interpretation.


In [61]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

#defining the models
models = {
    "RandomForest": RandomForestClassifier(random_state = 42, n_estimators = 300, n_jobs = -1),
    "HistGradientBoosting": HistGradientBoostingClassifier (random_state = 42)
}

for name, model in models.items():
    pipe = Pipeline([
        ("prep", preprocessor),
        ("model", model)
    ])

    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test) 

    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, average="macro")
    cm = confusion_matrix(y_test, preds)
    report = classification_report(y_test, preds)

    print(f"\n{'='*60}")
    print(f"MODEL: {name}")
    print(f"Accuracy:   {acc:.4f} ({acc:.2%})")
    print(f"F1 (macro): {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"\nDetailed Classification Report:\n{report}")

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'Neutral'

## Baseline + model comparison strategy
We compare models using:
- **Dummy baseline** (most frequent)
- ≥2 predictive models
Primary selection metric: **macro-F1** (balances performance across Low/Medium/High).


In [ ]:
from sklearn.dummy import DummyClassifier

dummy = Pipeline([
    ("prep", preprocessor),
    ("model", DummyClassifier(strategy="most_frequent"))
])

dummy.fit(X_train, y_train)
preds = dummy.predict(X_test)

print("Dummy Accuracy:", accuracy_score(y_test, preds))
print("Dummy F1 (macro):", f1_score(y_test, preds, average="macro"))

Dummy Accuracy: 0.337
Dummy F1 (macro): 0.1680378957865869


In [ ]:
results = []

for name, model in models.items():
    pipe = Pipeline([
        ("prep", preprocessor),
        ("model", model)
    ])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, preds),
        "F1_macro": f1_score(y_test, preds, average="macro")
    })

pd.DataFrame(results)


,Model,Accuracy,F1_macro
0,RandomForest,0.337,0.336824
1,HistGradientBoosting,0.331,0.330906


In [ ]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {"acc": "accuracy", "f1_macro": "f1_macro"}

all_models = {
    "Dummy_most_frequent": DummyClassifier(strategy="most_frequent"),
    "RandomForest": RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
}

rows = []
for name, model in all_models.items():
    pipe = Pipeline([("prep", preprocessor), ("model", model)])
    out = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    rows.append({
        "Model": name,
        "CV_Acc_mean": np.mean(out["test_acc"]),
        "CV_Acc_std": np.std(out["test_acc"]),
        "CV_F1macro_mean": np.mean(out["test_f1_macro"]),
        "CV_F1macro_std": np.std(out["test_f1_macro"]),
    })

cv_results = pd.DataFrame(rows).sort_values("CV_F1macro_mean", ascending=False)
cv_results

## Model selection
Final model is chosen using **highest mean CV macro-F1**. This avoids picking a model that just got lucky on one train/test split.


In [ ]:
best_name = cv_results.iloc[0]["Model"]
print("Selected model:", best_name)

final_model = all_models[best_name]
final_pipe = Pipeline([("prep", preprocessor), ("model", final_model)])
final_pipe.fit(X_train, y_train)

preds = final_pipe.predict(X_test)

print("FINAL Test Accuracy:", accuracy_score(y_test, preds))
print("FINAL Test F1 (macro):", f1_score(y_test, preds, average="macro"))
print("\nClassification report:\n", classification_report(y_test, preds))

ConfusionMatrixDisplay.from_predictions(y_test, preds)
plt.title(f"Confusion Matrix - {best_name}")
plt.show()


## Interpretation + limitations
- The model identifies patterns associated with stress level in this dataset (predictive association).
- It does **not** prove causation.
- If the dataset is synthetic / simulated, generalisation to real employees is limited.
- Possible leakage variables must remain excluded to avoid inflated performance.
